In [1]:
import numpy as np
import pandas as pd
import xarray as xr

/home/rait/anaconda3/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [2]:
ds = xr.open_mfdataset("/home/rait/Datasets/air.1948.nc")
air = ds['air']
air

<xarray.DataArray 'air' (time: 366, level: 17, lat: 73, lon: 144)>
dask.array<shape=(366, 17, 73, 144), dtype=float32, chunksize=(366, 17, 73, 144)>
Coordinates:
  * level    (level) float32 1000.0 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 1948-01-01 1948-01-02 ... 1948-12-31
Attributes:
    long_name:     mean Daily Air temperature
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       NCEP Reanalysis Daily Averages
    level_desc:    Multiple levels
    statistic:     Mean
    parent_stat:   Individual Obs
    actual_range:  [152.57 317.6 ]
    valid_range:   [150. 350.]

In [3]:
opt = xr.DataArray(data = None)
opt.values = False
opt.attrs["nval_crit"] = 29
opt

/home/rait/anaconda3/lib/python3.7/site-packages/xarray/core/dataarray.py:219: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  elif isinstance(data, pd.Panel):


<xarray.DataArray ()>
array(False)
Attributes:
    nval_crit:  29

In [4]:
def calculate_monthly_values(x, arith, nDim, opt):
    '''Calculate monthly values [avg, sum, min, max] from high frequency temporal values.
    
    
    Parameters
    ----------
    x : Array containing the high frequency data. The x must have an associated time coordinate variable in units recognized by the cd_calendar function.

        The following array structures are supported. The dimension name 'time' is a place-holder. Any name can be used. 
        The nDim argument specifies the dimension number to be used.

            (time)                     # nDim=0
            (time,npts)
            (time,ny,nx)
            (time,nz,ny,nx)
            (time,ne,nz,ny,nx)         # nDim=0; added for NCL version 6.5.0
            (ne,time,nz,ny,nx)         # nDim=1
       
    arith : A scalar string value which specifies the operation to be performed. 
            Valid values are: "avg" [also, "ave"], "sum", "min", "max", "var", "std". 
            It is required that x have associated with it a coordinate variable named time (ie, x&time) where "time" is recognized by cd_calendar.
    
    nDim : The dimension of x on which to calculate the statistic. Currently, only nDim=0 or 1 is allowed.
    
    
    Returns
    -------
    objectDataArray : An array of the same rank as x. 

    '''
    
    # Get the rank of dataarray
    len_of_dim = x.sizes
    rank_of_x = len(len_of_dim)
    
    # Check if the rank of dim is valid or invalid; if invalid, exit the function
    if (rank_of_x > 5):
        print("calculate_monthly_values: rank = {} [only 5 dimensions or fewer supported]".format(rank_of_x))
        return None
    
    
    # Get the variable name for time dimension
    time = x.dims[nDim]
    
    # Check if the DataArray already has monthly values or not
    days = x.time.dt.day
    uniq = np.unique(days)
    if (len(uniq) == 1):
        print("Data already in monthly format, or invalid.")
        return None
    
    if (rank_of_x <= 4):
        no_of_time = len_of_dim[x.dims[0]]
    else:
        no_of_time = len_of_dim[x.dims[1]]
        
    
    # First Year, Last Year and the number of years present in the DataArray
    yr_strt = x.time.dt.year[0].item()
    yr_last = x.time.dt.year[-1].item()
    no_of_years = yr_last - yr_strt + 1
    
    # Number of months in a year
    no_of_months = 12
    
    # Calculate the number of days for each month
    
    # A days per month dictionary to support different calendar types
    dpm = {'noleap': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '365_day': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'standard': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'proleptic_gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'all_leap': [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '366_day': [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '360_day': [0, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]}
    
    # Check if the year is leap year or not
    def leap_year(year, calendar='standard'):
        """Determine if year is a leap year"""
        leap = False
        if ((calendar in ['standard', 'gregorian',
            'proleptic_gregorian', 'julian']) and
            (year % 4 == 0)):
            leap = True
            if ((calendar == 'proleptic_gregorian') and
                (year % 100 == 0) and
                (year % 400 != 0)):
                leap = False
            elif ((calendar in ['standard', 'gregorian']) and
                     (year % 100 == 0) and (year % 400 != 0) and
                     (year < 1583)):
                leap = False
        return leap
    
    
    # Days per month function
    def get_dpm(time, calendar='standard'):
        """
        return an array of days per month corresponding to the months provided in `months`

        """
        month_length = np.zeros(len(time), dtype=np.int)

        cal_days = dpm[calendar]

        for i, (month, year) in enumerate(zip(time.month, time.year)):
            month_length[i] = cal_days[month]
            if leap_year(year, calendar=calendar):
                month_length[i] += 1
        return month_length
    
    
    # Save the month length values in a DataArray
    month_length = xr.DataArray(get_dpm(x.time.to_index(), calendar='noleap'),
                            coords=[x.time], name='month_length')
    
    # Copy the original DataArray
    x_new = x.copy(data = x, deep = True)
    
    # Load the new DataArray into memory (necessary for chunked data)
    x_new.load()
    
    # Check opt and get the nval_crit value
    if (opt.item() == True):
        nCritical = opt.attrs['nval_crit']
        
        # Set the nan index
        nan_index = np.array([])
        for i in range(0,len(month_length)):
            if (month_length[i].item() < nCritical):
                nan_index = np.append(nan_index,i)
        
        
        # Replace values with NaN for monthly_length < nCritical :: as per NCL function
        if (rank_of_x < 5):
            for ind in nan_index:
                nan_val = int(ind)
                x_new[nan_val] = None
        
        elif (rank_of_x == 5):
            for ind in nan_index:
                nan_val = int(ind)
                x_new[:,nan_val,:,:,:] = None
   
    # Perform the Arithmetic Computations
    
    if (arith == "avg"):
        xReturn = x_new.resample(time = '1M').mean(time, skipna = False)

    elif (arith == "sum"):
        xReturn = x_new.resample(time = '1M').sum(time, skipna = False)

    elif (arith == "min"):
        xReturn = x_new.resample(time = '1M').min(time, skipna = False)

    elif (arith == "max"):
        xReturn = x_new.resample(time = '1M').max(time, skipna = False)

    elif (arith == "var"):
        xReturn = x_new.resample(time = '1M').var(time, skipna = False)

    elif (arith == "std"):
        xReturn = x_new.resample(time = '1M').std(time, skipna = False)

    else:
        print("Unsupported arithmetic {}".format(arith))
        return None
    
    # Add and copy the attributes and properties from original DataArray
    xReturn = xReturn.rename("xMon")
    xReturn.attrs = x.attrs
    xReturn.encoding = x.encoding

    xReturn.attrs["time"] = nDim
    xReturn.attrs["NCL_tag"] = "calculate_monthly_values: arith=" + arith
    
    
    return xReturn

In [5]:
result = calculate_monthly_values(air, "std", 0, opt)

In [6]:
result

<xarray.DataArray 'xMon' (time: 12, level: 17, lat: 73, lon: 144)>
array([[[[4.145321, ..., 4.145321],
         ...,
         [3.988414, ..., 3.988414]],

        ...,

        [[5.876138, ..., 5.876138],
         ...,
         [2.043312, ..., 2.043312]]],


       ...,


       [[[4.896616, ..., 4.896616],
         ...,
         [1.969773, ..., 1.969773]],

        ...,

        [[5.267418, ..., 5.267418],
         ...,
         [1.839359, ..., 1.839359]]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1948-01-31 1948-02-29 ... 1948-12-31
  * level    (level) float32 1000.0 925.0 850.0 700.0 ... 50.0 30.0 20.0 10.0
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Attributes:
    long_name:     mean Daily Air temperature
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       

In [7]:
result[:,12,12,12].values

array([3.7297616, 3.4383311, 2.744366 , 1.2757269, 1.2679447, 1.9829098,
       1.6773349, 1.2525936, 3.3176465, 2.1348646, 2.4382775, 2.3353655],
      dtype=float32)